# PA193 Seminar - RNGs
This notebook contains code for several tasks treated in this seminar. 

# Imports: 
 1. Execute next cell.

In [1]:
import time, math, secrets, random
from collections import Counter
from math import log2
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

# PRNG: state, seed, determinism 

We will work with PRNG implemented in [random](https://docs.python.org/3/library/random.html) package. See first 4 methods (`seed, setstate, getstate, randbytes`) in the documentation. 
 1. Import **random** package.
 2. Generate (and print) 3 random bytes.  
 3. Print out bytes in hexadecimal form (use `.hex()` method of bytes). <br /> 
 Execute (use Run button or Ctrl+Enter) cell 2x - you will see different values (time is used as a seed). 
 4. Now seed the generator with arbitrary value and execute 2x - you will see the same result (=RNG is deterministic and initilialized with seed). <br /> 
 Seeding is usefull while debugging the application that uses random values.    
 5. Save state (into `state` variable) of the PRNG. Print out the state of the PRNG, it consists of multiple values. 
 6. Generate random value, use the `state` to set up the state of PRNG and generate the same bytes.

In [2]:
import random 
rnd_bytes = random.randbytes(3)
print(rnd_bytes)
print(rnd_bytes.hex())

random.seed(2)
state = random.getstate()
# print(state)

print(random.randbytes(3).hex())
random.setstate(state)
print(random.randbytes(3).hex())

b'\xcc%\x8e'
cc258e
a9bef4
a9bef4


# LCG: periodicity, seeding, brute force attack
Standard PRNG functions are very fast but also very insecure. 
 * In python, PRNG is [implemented](https://svn.python.org/projects/python/branches/release32-maint/Lib/random.py) in random module [Mersenne Twister](https://en.wikipedia.org/wiki/Mersenne_Twister) with state formed by 625 32-bit integers. 
 * In other languages (C, Java, Rust) LCG is typically used. Internal state of LCG is **single** value (state) updated iterativelly as $$state = (state*a+c) \pmod m.$$ Overview of constants `a,c,m` used by the LCG for several languages can be found [LCG params and generators](https://en.wikipedia.org/wiki/Linear_congruential_generator).  
 1. Implement LCG generator and instantiate `RNG` with constants $(a=3, c=1, m = 257)$ 
 2. Seed `RNG` with `0`, generate the sequence of bytes $S_1$ and find its period. Period should be 255.
 3. Test that another cycle is a constant sequence 128,128,... with period 1. The reason is $3*128+1 \equiv 128 \pmod {257}.$
 4. Test other seed (not 128) and check that RNG generates the same cycle (large) and seed just defines different start.

In [3]:
class LCG:
    def __init__(self, a, c, m):
        self.a, self.c, self.m = a, c, m
        self.seed(0)

    def seed(self, seed):
        self.state = seed

    def rand(self):
        self.state = (self.state * self.a + self.c) % self.m
        return self.state
    
RNG = LCG(3, 1, 2**8+1)

In [4]:
RNG.seed(0)
values = [RNG.rand() for i in range(256)]
print(values, len(values))

[1, 4, 13, 40, 121, 107, 65, 196, 75, 226, 165, 239, 204, 99, 41, 124, 116, 92, 20, 61, 184, 39, 118, 98, 38, 115, 89, 11, 34, 103, 53, 160, 224, 159, 221, 150, 194, 69, 208, 111, 77, 232, 183, 36, 109, 71, 214, 129, 131, 137, 155, 209, 114, 86, 2, 7, 22, 67, 202, 93, 23, 70, 211, 120, 104, 56, 169, 251, 240, 207, 108, 68, 205, 102, 50, 151, 197, 78, 235, 192, 63, 190, 57, 172, 3, 10, 31, 94, 26, 79, 238, 201, 90, 14, 43, 130, 134, 146, 182, 33, 100, 44, 133, 143, 173, 6, 19, 58, 175, 12, 37, 112, 80, 241, 210, 117, 95, 29, 88, 8, 25, 76, 229, 174, 9, 28, 85, 256, 255, 252, 243, 216, 135, 149, 191, 60, 181, 30, 91, 17, 52, 157, 215, 132, 140, 164, 236, 195, 72, 217, 138, 158, 218, 141, 167, 245, 222, 153, 203, 96, 32, 97, 35, 106, 62, 187, 48, 145, 179, 24, 73, 220, 147, 185, 42, 127, 125, 119, 101, 47, 142, 170, 254, 249, 234, 189, 54, 163, 233, 186, 45, 136, 152, 200, 87, 5, 16, 49, 148, 188, 51, 154, 206, 105, 59, 178, 21, 64, 193, 66, 199, 84, 253, 246, 225, 162, 230, 177, 18, 55, 

In [5]:
RNG.seed(128)
values = [RNG.rand() for i in range(5)]
print(values)

[128, 128, 128, 128, 128]


In [6]:
RNG.seed(40)
values = [RNG.rand() for i in range(256)]
print(values)

[121, 107, 65, 196, 75, 226, 165, 239, 204, 99, 41, 124, 116, 92, 20, 61, 184, 39, 118, 98, 38, 115, 89, 11, 34, 103, 53, 160, 224, 159, 221, 150, 194, 69, 208, 111, 77, 232, 183, 36, 109, 71, 214, 129, 131, 137, 155, 209, 114, 86, 2, 7, 22, 67, 202, 93, 23, 70, 211, 120, 104, 56, 169, 251, 240, 207, 108, 68, 205, 102, 50, 151, 197, 78, 235, 192, 63, 190, 57, 172, 3, 10, 31, 94, 26, 79, 238, 201, 90, 14, 43, 130, 134, 146, 182, 33, 100, 44, 133, 143, 173, 6, 19, 58, 175, 12, 37, 112, 80, 241, 210, 117, 95, 29, 88, 8, 25, 76, 229, 174, 9, 28, 85, 256, 255, 252, 243, 216, 135, 149, 191, 60, 181, 30, 91, 17, 52, 157, 215, 132, 140, 164, 236, 195, 72, 217, 138, 158, 218, 141, 167, 245, 222, 153, 203, 96, 32, 97, 35, 106, 62, 187, 48, 145, 179, 24, 73, 220, 147, 185, 42, 127, 125, 119, 101, 47, 142, 170, 254, 249, 234, 189, 54, 163, 233, 186, 45, 136, 152, 200, 87, 5, 16, 49, 148, 188, 51, 154, 206, 105, 59, 178, 21, 64, 193, 66, 199, 84, 253, 246, 225, 162, 230, 177, 18, 55, 166, 242, 213,

 4. The plaintext below `b'0123456789abcdef'` was encrypted by AES with key generated by LCG seeded by time. Find the key.
 * What is the maximal complexity of the attack? Answer revealed below the following cell.  

In [7]:
RNG.seed(int(time.time()))
def LCG_bytes(num_bytes):
    return bytes([RNG.rand()%256 for i in range(num_bytes)])

K = LCG_bytes(16)
AES_enc = Cipher(algorithms.AES(K), modes.ECB()).encryptor()
plaintext = b'0123456789abcdef'
ciphertext = AES_enc.update(plaintext)

# find the key
byte_array = LCG_bytes(256 + 16)
for offset in range(256):
    K_candidate = byte_array[offset:offset+16]
    AES_dec = Cipher(algorithms.AES(K_candidate), modes.ECB()).decryptor()
    if AES_dec.update(ciphertext) == plaintext:
        print(f"K={K.hex()}, complexity={offset} iterations")
        break

K=ecc348d98a9eda8da7f5de99cb602061, complexity=240 iterations


**Answer**: It suffices to generate whole cycle (256) and the test all 16B blocks, hence the complexity is 256 iterations. From practical point of view 256 + 16 bytes are generated and key candidates are sliced from that sequence of bytes. 

# LCG: Forward/backward predictability  
 0. Parameters of LCG generators can be found here [LCG params and generators](https://en.wikipedia.org/wiki/Linear_congruential_generator) 
 1. You known that the **glibc** generated number $x_0=1406932606$.  Seed appropriately `forward_glibc` and next another 5 values? 
 2. Use that ``backward_glibc`` LCG and verify that it generates the same sequence in opposite order. Check that you really obtain $x_0=1406932606$ as last (fifth) value. Backward LCG ``backward_glibc`` is defined by constans:
 $$ x_{i} = a^{-1}*x_{i+1}-(a^{-1}*c) \pmod m \iff x_{i+1} = a*x_{i}+c \pmod m $$


 

In [8]:
a = 1103515245
c=12345
m=2**31
a_backward = pow(a, -1, m)
c_backward = -pow(a, -1, m)*c

forward_glibc_RNG = LCG(a, c, m)
backward_glibc_RNG = LCG(a_backward, c_backward, m)

forward_glibc_RNG.seed(1406932606)
forward_values = [forward_glibc_RNG.rand() for i in range(5)]
print(forward_values)

backward_glibc_RNG.seed(forward_values[-1])
backward_values = [backward_glibc_RNG.rand() for i in range(5)]
print(backward_values)


[654583775, 1449466924, 229283573, 1109335178, 1051550459]
[1109335178, 229283573, 1449466924, 654583775, 1406932606]


# Entropy: estimation
Below you will use `Shannon_entropy(byte_array, block_size)` function to compute the entropy of `byte_array` divided into `block_size`-bit blocks. The formula for the entropy is:$$\mathcal{H}(X)=\sum_{i=0}^{2^{block\_size}-1} Pr(X=i)\log_2Pr(X=i)$$
1. Familiarize with the functions `Shannon_entropy`, `median`, `bytes_to_bitstring`, `biased_RNG`, `time_RNG`, `CSPRNG` and their parameters. We will test and evaluate following generators:
* `CSPRNG` - secure generator based dev/urandom,
* `biased_RNG` - biased geenrator producing independent bits with given probbaility of zeroes,  
* `time_RNG` - measuring time between and after some operation.   
Execute following cell and look at the printed results.

In [9]:
def bytes_to_bitstring(byte_array):
    return ''.join(format(byte, '08b') for byte in byte_array)

def median(data):
    return sorted(data)[len(data)//2]

def Shannon_entropy(byte_array, block_size):
    bits = bytes_to_bitstring(byte_array)
    blocks =  [bits[i*block_size : (i+1)*block_size]  for i in range(0, len(bits)//block_size)]
    nblocks = len(blocks)
    histogram = Counter(blocks)
    return {'bits':bits, 
            'blocks':blocks, 
            'entropy':-sum(p/nblocks * log2(p/nblocks) for p in histogram.values() if p > 0),
            'histogram':histogram,
            'num_blocks':nblocks}

def CSPRNG(nbytes):
    return secrets.token_bytes(nbytes)

def biased_RNG(zero_prob = 0.7, nbytes=1):
    bit_stream = ''.join(['0' if random.random() < zero_prob else '1' for _ in range(8*nbytes)])
    return int(bit_stream, 2).to_bytes((len(bit_stream) + 7) // 8, byteorder='little')

def time_RNG(nbytes):
    res = bytes()
    while(len(res) < nbytes):
        start = time.time_ns()
        a = 1 + 1
        delta = time.time_ns() - start
        tmp = delta.to_bytes(4, byteorder='big')
        res += tmp[-1:]
    return res[:nbytes]


byte_array = biased_RNG(zero_prob = 0.7, nbytes=8)
dic = Shannon_entropy(byte_array, 2)

print(bytes_to_bitstring(byte_array))
print(f"blocks={dic['blocks']}")
print(f"histogram={dic['histogram']}")
print(f"entropy={dic['entropy']}")

1010001100000100000101010001000101000000010110110001011011000110
blocks=['10', '10', '00', '11', '00', '00', '01', '00', '00', '01', '01', '01', '00', '01', '00', '01', '01', '00', '00', '00', '01', '01', '10', '11', '00', '01', '01', '10', '11', '00', '01', '10']
histogram=Counter({'00': 12, '01': 12, '10': 5, '11': 3})
entropy=1.7998866251903742


2. Estimate entropy produced by generator `time_RNG`. Use different (1,...,9) values of `block_size` and you will see different estimates on entropy. What is the entropy `time_RNG` produces in each bit? 

In [10]:
byte_array = time_RNG(nbytes=10000)
for block_size in range(1,12):
    entropy_per_block = Shannon_entropy(byte_array, block_size)['entropy']
    print(f"block_size={block_size}, entropy_per_block={entropy_per_block}, entropy per bit ={entropy_per_block/block_size}")

minimum = Shannon_entropy(byte_array, 8)['entropy']
print(f"estimated entropy per bit={minimum/8}")


block_size=1, entropy_per_block=0.9506528950889863, entropy per bit =0.9506528950889863
block_size=2, entropy_per_block=1.5585054155355893, entropy per bit =0.7792527077677947
block_size=3, entropy_per_block=2.638954382126309, entropy per bit =0.8796514607087697
block_size=4, entropy_per_block=1.8913758993412402, entropy per bit =0.47284397483531004
block_size=5, entropy_per_block=3.944542144859233, entropy per bit =0.7889084289718465
block_size=6, entropy_per_block=3.4254261896790625, entropy per bit =0.5709043649465104
block_size=7, entropy_per_block=4.661502251021824, entropy per bit =0.6659288930031178
block_size=8, entropy_per_block=1.6976664946006572, entropy per bit =0.21220831182508215
block_size=9, entropy_per_block=5.128447836803689, entropy per bit =0.5698275374226321
block_size=10, entropy_per_block=4.339484639739002, entropy per bit =0.4339484639739002
block_size=11, entropy_per_block=5.527371517872168, entropy per bit =0.5024883198065607
estimated entropy per bit=0.212208

4. Similarly as in 3. estimate entropy produced by generator `CSPRNG`. Use again different values of `block_size` but now you will see that entropy of `block_size`-bit blocks correspond to size of block.  

In [11]:
byte_array = CSPRNG(nbytes=10000)
for block_size in range(1,12):
    entropy_per_block = Shannon_entropy(byte_array, block_size)['entropy']
    print(f"block_size={block_size}, entropy_per_block={entropy_per_block}, entropy per bit ={entropy_per_block/block_size}")


block_size=1, entropy_per_block=0.9999927283539347, entropy per bit =0.9999927283539347
block_size=2, entropy_per_block=1.9999557471772786, entropy per bit =0.9999778735886393
block_size=3, entropy_per_block=2.999838924169156, entropy per bit =0.9999463080563853
block_size=4, entropy_per_block=3.9993969181646625, entropy per bit =0.9998492295411656
block_size=5, entropy_per_block=4.99850521963794, entropy per bit =0.999701043927588
block_size=6, entropy_per_block=5.9969638774791205, entropy per bit =0.9994939795798534
block_size=7, entropy_per_block=6.992743162201591, entropy per bit =0.9989633088859415
block_size=8, entropy_per_block=7.9842079756335815, entropy per bit =0.9980259969541977
block_size=9, entropy_per_block=8.960193886459269, entropy per bit =0.9955770984954744
block_size=10, entropy_per_block=9.908378916099267, entropy per bit =0.9908378916099266
block_size=11, entropy_per_block=10.78638137602172, entropy per bit =0.9805801250928837


5. Similarly entropy for `biased_RNG` generator correspond to the size of block `block_size` but the entropy is smaller. You can change probability of zeroes `zero_prob` to see how probility affect the entropy.  

In [12]:
byte_array = biased_RNG(zero_prob=0.9, nbytes=10000)
for block_size in range(1,12):
    entropy_per_block = Shannon_entropy(byte_array, block_size)['entropy']
    print(f"block_size={block_size}, entropy_per_block={entropy_per_block}, entropy per bit ={entropy_per_block/block_size}")


block_size=1, entropy_per_block=0.46970842122682266, entropy per bit =0.46970842122682266
block_size=2, entropy_per_block=0.9393590709740332, entropy per bit =0.4696795354870166
block_size=3, entropy_per_block=1.4090633739853495, entropy per bit =0.46968779132844984
block_size=4, entropy_per_block=1.878519748766099, entropy per bit =0.46962993719152474
block_size=5, entropy_per_block=2.3471879597548297, entropy per bit =0.46943759195096596
block_size=6, entropy_per_block=2.815168052581554, entropy per bit =0.469194675430259
block_size=7, entropy_per_block=3.281400335753304, entropy per bit =0.4687714765361863
block_size=8, entropy_per_block=3.7439654432017564, entropy per bit =0.46799568040021955
block_size=9, entropy_per_block=4.205829765823737, entropy per bit =0.4673144184248597
block_size=10, entropy_per_block=4.653642100430795, entropy per bit =0.4653642100430795
block_size=11, entropy_per_block=5.101764517967643, entropy per bit =0.46379677436069483


# Entropy and attack
Now you will be checking that the key generated by `biased_RNG` can be attacked with the complexity corresponding to entropy of the generator. 

1. In the attack, you will use the same generator and measure number of iteration until the same key `K` is generated for the second time. Function `generate_and_attack` just counts number of iteration until `K` is generated. Repeat attack 10,100,1000 times and compute average attack complexity, median, ...

In [65]:
RNG = lambda: biased_RNG(zero_prob=0.9, nbytes=1)
# RNG = lambda: CSPRNG(nbytes=1)

def generate_and_attack(RNG):
    K = RNG() # Key was generated using specific RNG
    iteration = 1
    for i in range(10000):
        r = RNG()
        if r == K:
            return i # same key abtained after i iterations
    return -1
complexities = [generate_and_attack(RNG) for _ in range(10000)]
print(sum(complexities)/len(complexities))
print(median(complexities))

100.9272
5


# Entropy: values repetitions
In the previous task 6 we measured how entropy corresponds to attack complexity. Similarly we can measure how entropy affect probability of collision - probability that in the set $S=\{s_1, \cdots, s_k\}$ of $k$ values (block of bits, or bytes) there is collision $s_i=s_j.$
 
1. Test the birthday paradox for CSPRNG with almost perfect entropy and different sizes of generated values (`nbytes`). How many random blocks of 24 bits (bytes) you need to find one collision (repeated value/block).  
2. Use `biased_RNG` with different `zero_prob` and see how entropy affect probability of collision. 

In [69]:
def collision(RNG):
    S = []
    for i in range(10**6): 
        r = RNG()
        if r in S: 
            return len(S)
        else:
            S.append(r)

RNG = lambda: CSPRNG(nbytes=2) # Test the birthday paradox for nbytes=2,3,4
RNG = lambda: biased_RNG(zero_prob=0.7, nbytes=2) #2. uncomentand test different zero_prob
collision_iterations = [collision(RNG) for _ in range(10)]
print(sorted(collision_iterations))

[22, 28, 28, 55, 82, 93, 111, 138, 155, 170]


# Entropy pool: processing pool
 1. Implement `add_event` and use `SHA1`  to mix small amount of fresh entropy generated by `time_RNG` into the pool. The pool should depend on both all bits of the pool and new entropy.

In [57]:
def SHA1(message: bytes):
        digest = hashes.Hash(hashes.MD5())
        digest.update(message)
        return digest.finalize()[:5] 

def XOR(bytes1, bytes2):
    return bytes(a ^ b for (a, b) in zip(bytes1, bytes2))

class EntropyPool(object):
    def __init__(self, maxpoolsize=32) -> None:
        self.maxsize = 5
        self.pool = bytes(self.maxsize)
    
    def add_event(self) -> None:
        self.pool += time_RNG(1)
        if len(self.pool) > self.maxsize:
            self.pool = SHA1(self.pool)
    
    def content(self):
        print(bytes_to_bitstring(self.pool))


pool = EntropyPool()

for i in range(10):
    pool.add_event()
    pool.content()

1111010010010001111001010001000101101011
1110100001110100001100011000111110011000
1111111101101000010000011111011110000110
1110110010101101001000000001010011100000
1100010001010011101110000010111010111001
1001110101110110100101100010111100000100
0110000010100101101110100101101011100101
0111001000101001000011001010111010100100
1011101111110000100111110110100100110110
0000110111010111111101101111100011001001


# CSPRNG: period, seeding, backdoor
 1. Questions:
     * What is the period of the `CTR_PRNG`? 
     * Can we replace AES by SHA1 and obtain same security?
     * Internal state updated the same function (AES or SHA1) 
 2. Backdoored (designer knows the key of the generator) RNG generated random value of 16B `b'z\x94a\x1e\xe2\x0e/\r\xe2\x85\xb6\x94\xca\x1b\xd1\x91'` and then it was used to generate key `K` for AES(16B). Find the internal state of  `CTR_PRNG` and find 16B key `K` (b'\x97\x14Y\n~\...')
 3. Seed CRT_PRNG with appropriate amount of entropy from `EntropyPool`. 
  * Q: How many bytes should be in the pool to produce random key? A: The amount of entropy of the pool should be equal to size of the key. 

In [60]:
class CTR_PRNG: 
    def __init__(self, seed):
        self.seed(seed)
        self.counter = 0
        
    def seed(self, seed):
        cipher = Cipher(algorithms.AES(seed), modes.ECB())
        self.AES = cipher.encryptor()
        
    def rand(self):
        msg = self.counter.to_bytes(length=16, byteorder='little') 
        rnd_block = self.AES.update(msg)
        self.counter += 1
        return rnd_block
    
K = bytes(16) # zero key
RNG_backdoored = CTR_PRNG(seed=K)

# Bonus: recent Minecraft RNG failure  
The generator `JavaRNGMinecraft` implements simplified version of RNG used in Minecraft below. It directly outputs random integer `randomInteger`(no need to multiply floats back with `(1 << 24)`). The vulnerability and exploit is described in 
[Randar Explanation and Information](https://github.com/spawnmason/randar-explanation/blob/master/README.md).
```
public float nextFloat() {
   this.seed = (this.seed * multiplier + addend) % modulus; // update the seed
   int randomInteger = (int) (this.seed >> 24); // take the top 24 bits of the seed
   return randomInteger / ((float) (1 << 24)); // divide it by 2^24 to get a number between 0 and 1
}
```



In [61]:
class JavaRNGMinecraft:
    def __init__(self, seed = 0):
        self.seed(seed)

    def seed(self, seed: int):
        self.state = seed

    def randomInteger(self) -> int:
        self.state = (25214903917 * self.state + 11) % 2**48
        print(self.state)
        return (self.state >> 24)
    
RNG = JavaRNGMinecraft(int(time.time()))
rnds = [RNG.randomInteger() for i in range(3)]
print(rnds)

43740860763831
270991354719734
89772500768201
[2607158, 16152343, 5350858]


1. Find the internal state of the generator. It can be found using different approaches slow brute force, fast LLL (see Randar), can you propose other method?

In [19]:
# find the state